In [37]:
import os
import sys
import astroNN
from astroNN import datasets
from astroNN.apogee import allstar, visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
from astroNN.models import load_folder
from astroNN.apogee.chips import wavelength_solution, chips_split

from astropy.io import fits
from astropy.wcs import WCS
import tensorflow as tf
from tensorflow.keras import callbacks, datasets, initializers, layers, losses, models, optimizers, preprocessing, utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import traceback
import h5py as h5

%matplotlib inline
import matplotlib.ticker as ticker

In [2]:
geir_path = "/geir_data/scr/petershi" #should save work here 
sdssv_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star" #path for SDSSV spectra
sdssiv_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/redux/dr17/stars/apo25m"

xmatch_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/ipl-1_dr17_lite_230517_byYS.fits"
allstar_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/aspcap/dr17/synspec/allStar-dr17-synspec.fits"
catlist_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/allStar-TIC_v8-APOGEE-0.2.6-v6_0_9-1.0.fits"

In [3]:
h5_path = os.path.join(geir_path, "serialized", "Retrain Dataset v2.hdf5")
hf = h5.File(h5_path, 'r')

In [4]:
def getSpectrum(star, ind, mask_value):

    spec = np.array(star["FLUX"])
    spec_err = np.array(star["FLUX_ERR"])
    bitmask = np.array(star["BITMASK"])

    target_bit = [0, 1, 2, 3, 4, 5, 6, 7, 12]

    try:
        _normspec, _spec_err = apogee_continuum(spec, spec_err, 
                                                bitmask=bitmask, 
                                                target_bit=target_bit, 
                                                dr=17, mask_value=mask_value)

        return (_normspec, _spec_err)

    except Exception: # for linalgerror
        #print("LinAlgError encountered for " + str(ind))
        return None

In [5]:
def read_h5(h5):

    keys = list(h5.keys())
    length = len(keys)

    normspec_container = []
    normspec_err_container = []
    elem_container = []
    elem_errs_container = []

    # symbols = ['teff', 'logg', 'C', 'CI', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
    #            'Ca', 'Ti', 'TiII', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni']

    symbols = ["teff", "logg", "Fe", "Mg"]

    try:
        for key in tqdm(keys):

            valid_elems = []
            valid_elem_errs = []

            grp = h5[key]

            spec = grp["Spectra"]
            elems = grp["Elements"]
            elem_errs = grp["Element Errors"]

            container = getSpectrum(spec, key, 1.)

            if container is not None:
                normspec = container[0]
                normspec_err = container[1]

                normspec_container.append(normspec[0])
                normspec_err_container.append(normspec_err[0])

                for sym in symbols:
                    if sym == "P":
                        valid_elems.append(elems["C"][()])
                        valid_elem_errs.append(elem_errs["C"][()])
                    else:
                        valid_elems.append(elems[sym][()])
                        valid_elem_errs.append(elem_errs[sym][()])

                elem_container.append(valid_elems)
                elem_errs_container.append(valid_elem_errs)


                # print(normspec)
                # print(normspec_err)
                # print(valid_elems)
                # print(valid_elem_errs)
    except:
        print(traceback.format_exc())

    return normspec_container, normspec_err_container, elem_container, elem_errs_container

In [6]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

In [7]:
def encodeArray(arrarr, d, n):
    length = len(arrarr[0])
    rl = []
    P = getPositionEncoding(seq_len = length, d = d, n = n)

    for arr in tqdm(arrarr):
        vec = np.atleast_2d(arr).T
        vec = np.hstack([vec]*d)

        rl.append(np.multiply(vec, P))
    
    return np.array(rl, dtype="float32")

In [8]:
def constructDataset(x, y, length):

    xcont = []
    ycont = []

    for xi, yi in tqdm(zip(x, y)):
        size = len(xi)
        for end in range(length, size):
            start = end - length 

            xcont.append(xi[start:end])
            ycont.append(yi)
    
    return np.array(xcont, dtype="float32"), np.array(ycont, dtype="float32")

In [27]:
def noAttention():  
    he_norm_init = initializers.HeNormal()  

    input_ = layers.Input(shape = (200, 16, 1), name = "input layer")  
    conv1 = layers.Conv2D(filters=16, kernel_size=5, padding="valid", activation = "relu", kernel_initializer = he_norm_init)(input_)  
    pool1 = layers.MaxPooling2D(pool_size=[2, 2], strides=2)(conv1)  
    # conv2 = layers.Conv2D(filters=16, kernel_size=5, padding="valid", activation = "relu", kernel_initializer = he_norm_init)(pool1)  
    # pool2 = layers.MaxPooling2D(pool_size=[2, 2], strides = 2)(conv2)  
    
    pool1_flat = layers.Flatten()(pool1)  
    
    # Fully connected layer #1: Has 120 neurons  
    dense1 = layers.Dense(units=120, activation = "relu", kernel_initializer = he_norm_init)(pool1_flat)  
    # Fully connected layer #2: Has 100 neurons  
    dense2 = layers.Dense(units=100, activation = "relu", kernel_initializer = he_norm_init)(dense1)  
    
    output = layers.Dense(units=10, activation = "linear")(dense2)  
   
    model = models.Model(input_, output)  

    return model
    
noat = noAttention()

/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.


In [41]:
def Attention():  
    he_norm_init = initializers.HeNormal()  

    input_ = layers.Input(shape = (200, 16, 1), name = "input layer")  

    query_value_attention_seq = layers.Attention()(
    [input_, input_])

    # query_encoding = layers.GlobalAveragePooling2D()(
    # input_)

    # print(query_encoding.shape)
    
    # query_value_attention = layers.GlobalAveragePooling2D()(
    # query_value_attention_seq)

    # print(query_value_attention.shape)


    input_layer = layers.Concatenate()(
    [query_encoding, query_value_attention])

    print(input_layer.shape)

    flat0 = layers.Flatten()(input_layer)


    conv1 = layers.Conv1D(filters=16, kernel_size=5, padding="valid", activation = "relu", kernel_initializer = he_norm_init)(flat0)  
    pool1 = layers.MaxPooling2D(pool_size=[2, 2], strides=2)(conv1)  
    # conv2 = layers.Conv2D(filters=16, kernel_size=5, padding="valid", activation = "relu", kernel_initializer = he_norm_init)(pool1)  
    # pool2 = layers.MaxPooling2D(pool_size=[2, 2], strides = 2)(conv2)  
    
    pool1_flat = layers.Flatten()(pool1)  
    
    # Fully connected layer #1: Has 120 neurons  
    dense1 = layers.Dense(units=120, activation = "relu", kernel_initializer = he_norm_init)(pool2_flat)  
    # Fully connected layer #2: Has 100 neurons  
    dense2 = layers.Dense(units=100, activation = "relu", kernel_initializer = he_norm_init)(dense1)  
    
    output = layers.Dense(units=10, activation = "linear")(dense2)  
   
    model = models.Model(input_, output)  

    return model
    
at = Attention()

(None, 1, 1, 1)
(None, 1, 1, 1)
(None, 1, 1, 2)


ValueError: Input 0 of layer "conv1d_2" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 2)

In [9]:
P = getPositionEncoding(seq_len=7514, d=32, n=10000)
print(P)

[[ 0.00000000e+00  1.00000000e+00  0.00000000e+00 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 8.41470985e-01  5.40302306e-01  5.33168440e-01 ...  9.99999950e-01
   1.77827940e-04  9.99999984e-01]
 [ 9.09297427e-01 -4.16146837e-01  9.02130715e-01 ...  9.99999800e-01
   3.55655875e-04  9.99999937e-01]
 ...
 [ 5.21010782e-01 -8.53550095e-01  9.92119020e-01 ... -7.20408005e-01
   9.72483909e-01  2.32970054e-01]
 [-4.36734312e-01 -8.99590541e-01  9.06147226e-01 ... -7.20627289e-01
   9.72525323e-01  2.32797116e-01]
 [-9.92947894e-01 -1.18551592e-01  5.41098597e-01 ... -7.20846501e-01
   9.72566705e-01  2.32624170e-01]]


In [10]:
# plt.figure(figsize = (5, 40))
# plt.imshow(P)
# plt.colorbar()
# plt.show()


In [11]:
vec = np.ones(7514)
vec = np.atleast_2d(vec).T
vec = np.hstack([vec]*32)
vec.shape

(7514, 32)

In [12]:
np.multiply(vec, P).flatten().shape

(240448,)

In [13]:
x, x_err, y, y_err = read_h5(hf)

  0%|          | 0/98224 [00:00<?, ?it/s]

/tmp/ipykernel_31193/502915280.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_31193/502915280.py:4: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_31193/502915280.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  0%|          | 226/98224 [00:03<23:45, 68.74it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


  1%|          | 660/98224 [00:10<22:21, 72.71it/s]/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/astroNN/apogee/chips.py:252: RuntimeWarning: invalid value encountered in divide
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/astroNN/apogee/chips.py:253: RuntimeWarning: invalid value encountered in divide
  1%|          | 717/98224 [00:11<24:21, 66.74it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


...

100%|█████████▉| 98099/98224 [24:11<00:01, 64.07it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


100%|█████████▉| 98186/98224 [24:12<00:00, 60.73it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


100%|██████████| 98224/98224 [24:13<00:00, 67.59it/s]


In [14]:
x_arr = np.array(x, dtype="float32")
y_arr = np.array(y, dtype="float32")
xerr_arr = np.array(x_err, dtype="float32")
yerr_arr =  np.array(y_err, dtype="float32")

In [17]:
x_pos_emb = encodeArray(x_arr[:100], 16, 10000)

100%|██████████| 100/100 [00:00<00:00, 711.61it/s]


In [19]:
x_train, y_train = constructDataset(x_pos_emb, y_arr, 200)

100it [00:00, 226.42it/s]


In [24]:
# with open(os.path.join(geir_path, "serialized", "x positional embeddings 30k.pkl"), "wb") as f:
#     pickle.dump(x_pos_emb, f)

In [20]:
x_train = np.array(x_train, dtype="float32")

In [21]:
sys.getsizeof(x_train)/10**9

9.361920144

In [23]:
x_train.shape

(731400, 200, 16)